<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/B-Create_API_call_and_Connecting_to_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating of a Flask application showing Citibike stations


In this segment we will create a basic app, where we will connect to the Citibike database, and display the list of stations. 



In [ ]:
!pip install -U PyMySQL sqlalchemy flask pyngrok geopandas pygeos

## A Refresher 

Let's remember first how to get data from the database.

In [ ]:
# This code creates a connection to the database
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)

In [ ]:
sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017"

with engine.connect() as connection:
    stations = pd.read_sql(sql, con=connection)

stations.to_dict(orient='records')

In [ ]:
stations.plot(kind='scatter', x='lon', y='lat')

In [ ]:
# Dataset from NYC Open Data: https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas/cpf4-rkhq
!curl 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=GeoJSON' -o nyc-neighborhoods.geojson

# Load the shapefile
df_nyc = gpd.GeoDataFrame.from_file('nyc-neighborhoods.geojson')

# Create the map of NYC neighborhoods
nyc_map = df_nyc.plot(linewidth=0.5, color='White', edgecolor='Black', figsize=(7, 5))

# Plot the stations on top of the map
stations.plot(kind='scatter', x='lon', y='lat', s=1, ax = nyc_map)

In [ ]:
# The code below is purely optional. 
# For comparison, let's get the current data from the Citibike API
import requests

# This gives information for each station that remains stable over time
url_stations = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json()

# Put the results from the Citibike API in a dataframe
df = pd.DataFrame(results['data']['stations'])

# Remove the noisy lon/lat points with coordinates (0,0) 
df = df.query("lon!=0  and lat!=0")

# Create the map of NYC neighborhoods
nyc_map = df_nyc.plot(linewidth=0.5, color='White', edgecolor='Black', figsize=(7, 5))

# Plot the stations on top of the map
df.plot(kind='scatter', x='lon', y='lat', s=1, ax = nyc_map)

## Creating an API endpoint

In [ ]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

from flask import render_template, jsonify
from sqlalchemy import create_engine
import pandas as pd
import base64
from io import BytesIO
import matplotlib.pyplot as plt

In [ ]:
# Setup Flask and ngrok

os.environ["FLASK_DEBUG"] = "true"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
ngrok_authtoken = '2EYf3qVk9mi739HjPwSNZXWAtfy_4jF9NAhGqVVVJmm4YehPW'
ngrok.set_auth_token(ngrok_authtoken)
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url


In [ ]:
# Setup a connection to the database
conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')
engine = create_engine(conn_string)

#### Now let's start our server:

A few things to notice.

First, notice the `@app.route('/citibike_api',  methods=['GET'])` command. This part specifies that our API endpoint will be accessible under the `http://our_web_server_address/citibike_api`.


Then, we define the function `def citibike_stations():` that will create the response of that API call. What the function returns is what the API call will return back.

Notice that insider the `citibike_stations` function, we connect to the database, and issue an SQL query to the database.

Finally, we get back the results of the query, we put the results in a Python dictionary, and we use the `jsonify` function to convert our dictionary to JSON and return it as the API result.


In [ ]:
# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"

In [ ]:
@app.route('/citibike_api',  methods=['GET'])
def citibike_stations():

    sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
    # Connect to the database, execute the query, and get back the results
    with engine.connect() as connection:
        stations = pd.read_sql(sql, con=connection)
    
    # Create the response. We will put the retrieved data as a list of
    # dictionaries, under the key "stations".
    list_of_stations = stations.to_dict(orient='records')
    
    api_results = {"stations": list_of_stations}

    # We JSON-ify our dictionary and return it as the API response
    return jsonify(api_results)

The next api call `/station_map` is a bit different. Instead of returning back the raw data, creates a plot, and then returns the image data in the JSON response. The image data is encoded in "base64" format. We will examine how we can use such API calls to serve dynamically generated images for our website.

In [ ]:
@app.route('/station_map',  methods=['GET'])
def station_map():


    # Connect to the database, execute the query, and get back the results
    sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
    with engine.connect() as connection:  
        stations = pd.read_sql(sql, con=connection)

    fig, ax = plt.subplots()
    ax = stations.plot(kind='scatter', x='lon', y='lat', ax=ax)

    buf = BytesIO()
    fig.savefig(buf, format="png")
    # Embed the result in the html output.
    data = base64.b64encode(buf.getbuffer()).decode("ascii")

    # Create the response. We will put the retrieved data as a list of
    # dictionaries, under the key "stations".
    results = {"image": data}

    # We JSON-ify our dictionary and return it as the API response
    return jsonify(results)

    # return f"<img src='data:image/png;base64,{data}'/>"
 


In [ ]:
# Once you run your app, check the /citibike_api and the /station_map API calls

print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")
app.run(use_reloader=False, port=port)

## Exercise

a. Connect to a database of your choice and create an API call that returns the results from the database in JSON format.

b. Using the results from your query is part (a), create a plot and create an API call that return the "base64" of the plot.